In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

# Load dataset
path = ""C:\Users\praanand\Downloads\sp500_20years Ra Dataset (1).csv""

df = pd.read_csv(path, skiprows=1)
df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values('Date', inplace=True)
close_prices = df['Close'].values

# Prepare data (lag=1 for autoregression)
X = close_prices[:-1].reshape(-1, 1)
y = close_prices[1:].reshape(-1, 1)

# Simple Neural Network (1 hidden layer)
class NeuralNetwork:
    def __init__(self, w1, b1, w2, b2):
        self.w1 = w1  # Input -> Hidden weights
        self.b1 = b1  # Hidden bias
        self.w2 = w2  # Hidden -> Output weights
        self.b2 = b2  # Output bias

    def forward(self, x):
        self.h = np.maximum(0, x @ self.w1 + self.b1)  # ReLU activation
        return self.h @ self.w2 + self.b2

    def train(self, X, y, lr=0.001, epochs=10):
        for _ in range(epochs):
            # Forward pass
            y_pred = self.forward(X)

            # Backpropagation
            grad_y = 2 * (y_pred - y) / len(y)
            grad_w2 = self.h.T @ grad_y
            grad_b2 = grad_y.sum(axis=0)

            grad_h = grad_y @ self.w2.T
            grad_h[self.h <= 0] = 0  # ReLU derivative
            grad_w1 = X.T @ grad_h
            grad_b1 = grad_h.sum(axis=0)

            # Update weights
            self.w1 -= lr * grad_w1
            self.b1 -= lr * grad_b1
            self.w2 -= lr * grad_w2
            self.b2 -= lr * grad_b2

# Generate 20 random initial conditions
np.random.seed(42)
initial_conditions = [
    (
        np.random.randn(1, 10),  # w1
        np.random.randn(1, 10),  # b1
        np.random.randn(10, 1),  # w2
        np.random.randn(1, 1)    # b2
    ) for _ in range(20)
]

# Entropy calculation remains the same
def calculate_entropy(data, bins=20):
    counts, _ = np.histogram(data, bins=bins)
    prob = counts / counts.sum()
    return entropy(prob)

# Evaluate initial conditions
entropies = []
for w1, b1, w2, b2 in initial_conditions:
    model = NeuralNetwork(w1, b1, w2, b2)
    model.train(X, y, epochs=50, lr=0.0001)
    y_pred = model.forward(X)
    errors = y - y_pred.flatten()
    entropies.append(calculate_entropy(errors))

# Rank and display results
ranked_conditions = sorted(zip(initial_conditions, entropies), key=lambda x: x[1])
print("Top 5 Neural Network Initializations:")
for (w1, b1, w2, b2), e in ranked_conditions[:5]:
    print(f"Entropy: {e:.4f} | Shape: w1{w1.shape}, w2{w2.shape}")

Top 5 Neural Network Initializations:
Entropy: 2.6377 | Shape: w1(1, 10), w2(10, 1)
Entropy: 2.6377 | Shape: w1(1, 10), w2(10, 1)
Entropy: 2.6377 | Shape: w1(1, 10), w2(10, 1)
Entropy: 2.6377 | Shape: w1(1, 10), w2(10, 1)
Entropy: 2.6377 | Shape: w1(1, 10), w2(10, 1)


In [ ]:
!pip install nlopt
import nlopt

# Load dataset
# Prepare data (lag=1 autoregression)
X = close_prices[:-1].reshape(-1, 1)
y = close_prices[1:].reshape(-1, 1)

# Neural Network with NLOPT optimization
class NeuralNetwork:
    def __init__(self, params):
        self.w1, self.b1, self.w2, self.b2 = self.unflatten_params(params)

    def unflatten_params(self, params):
        w1 = params[0:10].reshape(1, 10)
        b1 = params[10:20].reshape(1, 10)
        w2 = params[20:30].reshape(10, 1)
        b2 = params[30:31].reshape(1, 1)
        return w1, b1, w2, b2

    def forward(self, x):
        h = np.maximum(0, x @ self.w1 + self.b1)
        return h @ self.w2 + self.b2

# Generate 20 random initial guesses
np.random.seed(42)
initial_guesses = [np.random.randn(31) for _ in range(20)]  # 31 total parameters

# NLOPT optimization setup
def objective_function(params, grad):
    model = NeuralNetwork(params)
    y_pred = model.forward(X)
    errors = y.flatten() - y_pred.flatten()
    return entropy(np.histogram(errors, bins=20)[0])

improved_conditions = []
for init_params in initial_guesses:
    opt = nlopt.opt(nlopt.LN_BOBYQA, len(init_params))
    opt.set_min_objective(objective_function)
    opt.set_xtol_rel(1e-4)
    opt.set_maxeval(100)  # Optimization budget

    try:
        optimized_params = opt.optimize(init_params.copy())
        improved_conditions.append(optimized_params)
    except:
        improved_conditions.append(init_params)  # Fallback

# Verify improvement
original_entropy = [objective_function(p, None) for p in initial_guesses]
optimized_entropy = [objective_function(p, None) for p in improved_conditions]

print(f"Average entropy reduction: {np.mean(original_entropy) - np.mean(optimized_entropy):.2f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 6.7 MB/s eta 0:00:00
Average entropy reduction: 1.08


In [ ]:
import pandas as pd
import numpy as np
import nlopt
from scipy.stats import entropy

returns = np.log(close_prices[1:]/close_prices[:-1])  # Log returns

class ARMA_GARCH_NN:
    def __init__(self, params):
        # ARMA-GARCH(1,1) parameters
        self.phi, self.theta, self.omega, self.alpha, self.beta = params[:5]

        # Neural Network parameters
        nn_params = params[5:]
        self.w1 = nn_params[0:10].reshape(1, 10)
        self.b1 = nn_params[10:20].reshape(1, 10)
        self.w2 = nn_params[20:30].reshape(10, 1)
        self.b2 = nn_params[30:31].reshape(1, 1)

    def compute_residuals(self, returns):
        # ARMA(1,1) residuals
        eps = np.zeros_like(returns)
        for t in range(1, len(returns)):
            eps[t] = returns[t] - (self.phi*returns[t-1] + self.theta*eps[t-1])

        # GARCH(1,1) variances
        sigma2 = np.zeros_like(returns)
        sigma2[0] = np.var(returns)
        for t in range(1, len(returns)):
            sigma2[t] = self.omega + self.alpha*eps[t-1]**2 + self.beta*sigma2[t-1]

        return eps/np.sqrt(sigma2[1:])  # Standardized residuals

    def nn_forward(self, x):
        h = np.maximum(0, x @ self.w1 + self.b1)
        return h @ self.w2 + self.b2

# NLOPT objective function
def objective(params, grad):
    model = ARMA_GARCH_NN(params)
    try:
        # Get GARCH-standardized residuals
        residuals = model.compute_residuals(returns)

        # Neural network prediction on residuals
        X = residuals[:-1].reshape(-1, 1)
        y = residuals[1:].reshape(-1, 1)
        y_pred = model.nn_forward(X)

        errors = y.flatten() - y_pred.flatten()
        return entropy(np.histogram(errors, bins=20)[0])
    except:
        return float('inf')  # Handle unstable parameters

# Generate 20 initial guesses (ARMA-GARCH + NN params)
np.random.seed(42)
initial_conditions = []
for _ in range(20):
    # ARMA-GARCH parameters (stationary initialization)
    arma_garch = [
        np.random.uniform(-0.9, 0.9),  # phi
        np.random.uniform(-0.9, 0.9),  # theta
        np.random.uniform(0, 0.1),     # omega
        np.random.uniform(0, 0.3),     # alpha
        np.random.uniform(0.5, 0.9)    # beta
    ]
    # NN parameters
    nn_params = np.random.randn(31)
    initial_conditions.append(np.concatenate([arma_garch, nn_params]))

# Optimization setup
improved_conditions = []
for init in initial_conditions:
    opt = nlopt.opt(nlopt.LN_BOBYQA, len(init))
    opt.set_min_objective(objective)
    opt.set_lower_bounds([-0.99, -0.99, 1e-6, 1e-6, 0.5] + [-np.inf]*31)
    opt.set_upper_bounds([0.99, 0.99, 0.1, 0.3, 0.99] + [np.inf]*31)
    opt.set_xtol_rel(1e-4)
    opt.set_maxeval(150)

    try:
        optimized = opt.optimize(init.copy())
        improved_conditions.append(optimized)
    except:
        improved_conditions.append(init)

# Evaluate improvement
original_scores = [objective(init, None) for init in initial_conditions]
optimized_scores = [objective(opt, None) for opt in improved_conditions]

print(f"Average entropy reduction: {np.mean(original_scores) - np.mean(optimized_scores):.2f} bits")
print("Best optimized parameters:")
print("ARMA-GARCH:", improved_conditions[0][:5])
print("NN shapes:", (1,10), (10,1))

Average entropy reduction: nan bits
Best optimized parameters:
ARMA-GARCH: [-0.22582779  0.81128575  0.07319939  0.17959755  0.56240746]
NN shapes: (1, 10) (10, 1)


<ipython-input-3-d80642500f4a>:91: RuntimeWarning: invalid value encountered in scalar subtract
  print(f"Average entropy reduction: {np.mean(original_scores) - np.mean(optimized_scores):.2f} bits")


In [ ]:
import pandas as pd
import numpy as np
import nlopt
from scipy.stats import t
from scipy.stats import kurtosis  # Import the correct function

class TDist_ARMA_GARCH_NN:
    def __init__(self, params):
        # ARMA-GARCH(1,1) + Student's T parameters
        self.phi, self.theta, self.omega, self.alpha, self.beta, self.nu = params[:6]

        # Neural Network parameters
        nn_params = params[6:]
        self.w1 = nn_params[0:10].reshape(1, 10)
        self.b1 = nn_params[10:20].reshape(1, 10)
        self.w2 = nn_params[20:30].reshape(10, 1)
        self.b2 = nn_params[30:31].reshape(1, 1)

    def compute_std_residuals(self, returns):
        # ARMA(1,1) residuals
        eps = np.zeros_like(returns)
        for t in range(1, len(returns)):
            eps[t] = returns[t] - (self.phi*returns[t-1] + self.theta*eps[t-1])

        # GARCH(1,1) variances
        sigma2 = np.zeros_like(returns)
        sigma2[0] = np.var(returns)
        for t in range(1, len(returns)):
            sigma2[t] = self.omega + self.alpha*eps[t-1]**2 + self.beta*sigma2[t-1]

        # Fix: Align dimensions by slicing both arrays from index 1
        return eps[1:]/np.sqrt(sigma2[1:]), sigma2[1:]

    def log_likelihood(self, residuals, sigma2):
        # Student's T log-likelihood
        return t.logpdf(residuals, df=self.nu, loc=0, scale=np.sqrt(sigma2*(self.nu-2)/self.nu)).sum()

    def nn_forward(self, x):
        h = np.maximum(0, x @ self.w1 + self.b1)
        return h @ self.w2 + self.b2

# NLOPT objective function (negative log-likelihood)
def objective(params, grad):
    model = TDist_ARMA_GARCH_NN(params)
    try:
        residuals, sigma2 = model.compute_std_residuals(returns)
        ll = model.log_likelihood(residuals, sigma2)
        return -ll  # Minimize negative log-likelihood
    except:
        return float('inf')

# Generate 20 initial guesses with T-distribution parameters
np.random.seed(42)
initial_conditions = []
for _ in range(20):
    # ARMA-GARCH-T parameters
    base_params = [
        np.random.uniform(-0.9, 0.9),   # phi
        np.random.uniform(-0.9, 0.9),   # theta
        np.random.uniform(0, 0.1),      # omega
        np.random.uniform(0, 0.3),      # alpha
        np.random.uniform(0.5, 0.9),    # beta
        np.random.uniform(3, 7)         # nu (degrees of freedom)
    ]
    # NN parameters
    nn_params = np.random.randn(31)
    initial_conditions.append(np.concatenate([base_params, nn_params]))

# Optimization setup with bounds
improved_conditions = []
for init in initial_conditions:
    opt = nlopt.opt(nlopt.LN_BOBYQA, len(init))
    opt.set_min_objective(objective)
    opt.set_lower_bounds([-0.99, -0.99, 1e-6, 1e-6, 0.5, 2.1] + [-np.inf]*31)
    opt.set_upper_bounds([0.99, 0.99, 0.1, 0.3, 0.99, 15] + [np.inf]*31)
    opt.set_xtol_rel(1e-4)
    opt.set_maxeval(200)

    try:
        optimized = opt.optimize(init.copy())
        improved_conditions.append(optimized)
    except:
        improved_conditions.append(init)

# Analyze results
best_params = improved_conditions[np.argmin([objective(p, None) for p in improved_conditions])]
model = TDist_ARMA_GARCH_NN(best_params)
residuals, _ = model.compute_std_residuals(returns)

# Sample kurtosis (Fisher-Pearson, excess)
sample_kurt = kurtosis(residuals, fisher=True)

# Theoretical kurtosis for Student's T (if nu > 4)
nu = best_params[5]
theoretical_kurt = 6/(nu - 4) if nu > 4 else float('inf')

print(f"Residuals kurtosis: Sample={sample_kurt:.2f}, Theoretical={theoretical_kurt:.2f} (Normal=0)")

Residuals kurtosis: Sample=11.56, Theoretical=inf (Normal=0)


In [ ]:
import pandas as pd
import numpy as np
import nlopt
from scipy.stats import t, kurtosis

def hurst_rogers(series):
    """Calculate Hurst exponent using Rogers' modified R/S method"""
    n = len(series)
    max_lag = min(100, n//2)
    lags = np.arange(2, max_lag)
    rs_values = []

    for lag in lags:
        chunks = [series[i:i+lag] for i in range(0, n, lag)]
        valid_chunks = [c for c in chunks if len(c) >= lag]

        chunk_rs = []
        for chunk in valid_chunks:
            mean_chunk = np.mean(chunk)
            adjusted = chunk - mean_chunk
            cumulative = np.cumsum(adjusted)
            r = np.max(cumulative) - np.min(cumulative)
            s = np.std(chunk, ddof=1)
            if s != 0:
                chunk_rs.append(r/s)

        if chunk_rs:
            rs_values.append(np.mean(chunk_rs))

    if not rs_values:
        return 0.5  # Fallback if no valid RS values

    hurst = np.polyfit(np.log(lags[:len(rs_values)]), np.log(rs_values), 1)[0]
    return hurst

class AdvancedRiskModel:
    def __init__(self, params):
        # Model parameters
        self.phi, self.theta = params[:2]
        self.omega, self.alpha, self.beta, self.nu = params[2:6]
        self.nn_params = params[6:]
        self.hurst_weight = 0.5  # Weight for Hurst-based penalty

    def compute_std_residuals(self, returns):
        """Calculate standardized residuals using ARMA-GARCH model"""
        n = len(returns)
        epsilon = np.zeros(n)  # Residuals (ε)
        sigma2 = np.zeros(n)   # Conditional variance (σ²)

        # Initialize with first observation
        epsilon[0] = returns[0]
        sigma2[0] = np.var(returns)

        for t in range(1, n):
            # ARMA(1,1) component
            arma_mean = self.phi * returns[t-1] + self.theta * epsilon[t-1]
            epsilon[t] = returns[t] - arma_mean

            # GARCH(1,1) component
            sigma2[t] = self.omega + self.alpha * epsilon[t-1]**2 + self.beta * sigma2[t-1]

        # Standardized residuals (ε_t/σ_t)
        std_residuals = epsilon / np.sqrt(sigma2)
        return std_residuals, sigma2

    def log_likelihood(self, std_residuals, sigma2):
        """Calculate log-likelihood for Student's t-distribution"""
        return np.sum(t.logpdf(std_residuals, df=self.nu) - 0.5 * np.log(sigma2))

    def compute_hurst_penalty(self, residuals):
        """Hurst-based regularization penalty"""
        h_residuals = hurst_rogers(residuals)
        return self.hurst_weight * (h_residuals - 0.5)**2

    def objective_function(self, params, grad=None):
        # Update parameters
        self.phi, self.theta = params[:2]
        self.omega, self.alpha, self.beta, self.nu = params[2:6]
        self.nn_params = params[6:]

        # Compute residuals and likelihood
        residuals, sigma2 = self.compute_std_residuals(returns)
        ll = self.log_likelihood(residuals, sigma2)

        # Hurst regularization
        hurst_penalty = self.compute_hurst_penalty(residuals)

        return -ll + hurst_penalty

# Generate synthetic returns data (replace with real data)
np.random.seed(42)
returns = pd.Series(np.random.randn(1000) * 0.02)

# Parameter initialization
initial_params = np.array([
    0.0, 0.0,       # ARMA: phi, theta
    0.05, 0.1, 0.8, # GARCH: omega, alpha, beta
    4.0,             # Student's t ν (degrees of freedom)
    *np.random.randn(31) * 0.01  # Neural network parameters
])

# Initial Hurst analysis
H_original = hurst_rogers(returns)
print(f"Original Hurst exponent: {H_original:.2f}")

# Optimization setup
opt = nlopt.opt(nlopt.LN_BOBYQA, len(initial_params))
opt.set_min_objective(AdvancedRiskModel(initial_params).objective_function)
opt.set_lower_bounds([-0.99, -0.99, 1e-6, 1e-6, 0.5, 2.1] + [-np.inf]*31)
opt.set_upper_bounds([0.99, 0.99, 0.1, 0.3, 0.99, 15] + [np.inf]*31)
opt.set_ftol_rel(1e-6)  # Convergence tolerance

# Run optimization
optimized_params = opt.optimize(initial_params.copy())

# Final analysis
final_model = AdvancedRiskModel(optimized_params)
residuals, _ = final_model.compute_std_residuals(returns)
H_final = hurst_rogers(residuals)

print("\nOptimization Results:")
print(f"Hurst Exponent - Original: {H_original:.2f}, Residuals: {H_final:.2f}")
print(f"Volatility Persistence (α+β): {final_model.alpha + final_model.beta:.2f}")
print(f"Tail Risk (ν): {final_model.nu:.1f} degrees of freedom")

Original Hurst exponent: 0.64

Optimization Results:
Hurst Exponent - Original: 0.64, Residuals: 0.65
Volatility Persistence (α+β): 0.77
Tail Risk (ν): 2.6 degrees of freedom


In [ ]:
# Importance Sampling for Initial Conditions
import numpy as np
from scipy.stats import t  # Student's T distribution for likelihood calculations

# Ensure returns data is available (log returns of close_prices)
returns = np.log(close_prices[1:] / close_prices[:-1])

# **Step 1:** Generate 20 initial condition guesses (proposal distribution)
np.random.seed(42)
initial_conditions = []
for _ in range(20):
    # ARMA-GARCH-T parameters (within plausible bounds for stationarity and nu > 2)
    base_params = [
        np.random.uniform(-0.9, 0.9),    # phi (AR coefficient)
        np.random.uniform(-0.9, 0.9),    # theta (MA coefficient)
        np.random.uniform(0, 0.1),       # omega (GARCH constant)
        np.random.uniform(0, 0.3),       # alpha (GARCH alpha)
        np.random.uniform(0.5, 0.9),     # beta (GARCH beta)
        np.random.uniform(3, 7)          # nu (degrees of freedom for Student's T)
    ]
    # Neural network weights (proposal: standard normal)
    nn_params = np.random.randn(31)
    initial_conditions.append(np.concatenate([base_params, nn_params]))

initial_conditions = np.array(initial_conditions)  # shape (20, 37)

# **Step 2:** Define a function to compute log-likelihood for a given parameter set
def compute_log_likelihood(params):
    """
    Compute the log-likelihood of the returns data under the ARMA(1,1)-GARCH(1,1)-T model
    with a neural network mean adjustment, given a parameter vector.
    Returns -inf if the parameters produce an invalid model (e.g., non-positive variance).
    """
    try:
        # Instantiate the model using the previously defined TDist_ARMA_GARCH_NN class
        model = TDist_ARMA_GARCH_NN(params)
        # Compute standardized residuals and conditional variances
        residuals, sigma2 = model.compute_std_residuals(returns)
        # Sum log-likelihood of Student's T for all data points
        ll = model.log_likelihood(residuals, sigma2)
        return ll
    except Exception as e:
        # If any numerical issue occurs (e.g., non-positive variance), treat log-likelihood as -inf
        return -np.inf

# **Step 3:** Evaluate log-likelihoods for all initial guesses and compute importance weights
log_liks = np.array([compute_log_likelihood(p) for p in initial_conditions])
# Identify the highest log-likelihood among the samples (for numerical stability)
max_log_lik = np.max(log_liks[np.isfinite(log_liks)]) if np.any(np.isfinite(log_liks)) else -np.inf

# Compute unnormalized weights proportional to likelihood = exp(log_lik)
# Use relative scaling by subtracting max_log_lik to avoid underflow for very low likelihoods
unnormalized_weights = np.exp(log_liks - max_log_lik)
# Any -inf log_lik becomes weight 0. If all log_liks are -inf (unlikely with broad initial search), use equal weights
if not np.any(np.isfinite(log_liks)):
    unnormalized_weights = np.ones_like(unnormalized_weights)
# Normalize weights to form a probability distribution
weights = unnormalized_weights / unnormalized_weights.sum()

# **Step 4:** Resample 20 new initial conditions based on the importance weights
# This concentrates samples in regions with higher likelihood (lower risk modeling error)
resample_indices = np.random.choice(len(initial_conditions), size=len(initial_conditions), p=weights)
refined_conditions = initial_conditions[resample_indices]

# **Step 5:** (Optional) Evaluate improvement in objective metric for the refined guesses
orig_costs = np.array([objective(p, None) for p in initial_conditions])      # original negative log-likelihoods
refined_costs = np.array([objective(p, None) for p in refined_conditions])   # refined negative log-likelihoods

# Filter out any infinite costs for averaging
orig_fin = orig_costs[np.isfinite(orig_costs)]
ref_fin = refined_costs[np.isfinite(refined_costs)]
avg_orig = np.mean(orig_fin) if orig_fin.size > 0 else np.inf
avg_refined = np.mean(ref_fin) if ref_fin.size > 0 else np.inf

print(f"Average negative log-likelihood: Before = {avg_orig:.2f}, After = {avg_refined:.2f}")


Average negative log-likelihood: Before = -1112.25, After = -4961.98
